In [ ]:
#prototype 

import cv2

vid = cv2.VideoCapture(0)

while True:
        try:
            res,frame = vid.read()
            cv2.imshow("Video",frame)
        
        except:
            pass
        if cv2.waitKey(1) & 0xff==ord('l'):
            break
    
cv2.destroyAllWindows()

In [ ]:
#prototype 

import cv2
import mediapipe as mp

# Initialize Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip frame for natural viewing
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process with FaceMesh
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Draw face mesh for visualization
            mp_drawing.draw_landmarks(
                frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS,
                mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1),
                mp_drawing.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1)
            )

            # Eye landmarks (Iris tracking)
            left_iris = face_landmarks.landmark[468]   # left eye iris center
            right_iris = face_landmarks.landmark[473]  # right eye iris center
            nose_tip = face_landmarks.landmark[1]      # nose tip (for reference)

            h, w, _ = frame.shape
            left_x, left_y = int(left_iris.x * w), int(left_iris.y * h)
            right_x, right_y = int(right_iris.x * w), int(right_iris.y * h)
            nose_x, nose_y = int(nose_tip.x * w), int(nose_tip.y * h)

            # Draw points
            cv2.circle(frame, (left_x, left_y), 3, (255, 0, 0), -1)
            cv2.circle(frame, (right_x, right_y), 3, (255, 0, 0), -1)
            cv2.circle(frame, (nose_x, nose_y), 3, (0, 255, 255), -1)

            # Simple heuristic: check if iris is aligned with nose (looking forward)
            eye_center_x = (left_x + right_x) // 2
            if abs(eye_center_x - nose_x) < 20:  
                cv2.putText(frame, "Eye Contact: YES", (30, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Eye Contact: NO", (30, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Eye Contact Tracker", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#prototype 

import cv2
import mediapipe as mp
import numpy as np

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# Eye landmarks (from Mediapipe FaceMesh)
LEFT_EYE = [33, 160, 158, 133, 153, 144]   # left eye region
RIGHT_EYE = [362, 385, 387, 263, 373, 380] # right eye region

def eye_aspect_ratio(landmarks, eye_points, w, h):
    # Get landmark coordinates
    p1 = np.array([landmarks[eye_points[1]].x * w, landmarks[eye_points[1]].y * h])
    p2 = np.array([landmarks[eye_points[2]].x * w, landmarks[eye_points[2]].y * h])
    p3 = np.array([landmarks[eye_points[5]].x * w, landmarks[eye_points[5]].y * h])
    p4 = np.array([landmarks[eye_points[4]].x * w, landmarks[eye_points[4]].y * h])
    p0 = np.array([landmarks[eye_points[0]].x * w, landmarks[eye_points[0]].y * h])
    p3b = np.array([landmarks[eye_points[3]].x * w, landmarks[eye_points[3]].y * h])

    # Compute EAR
    vertical1 = np.linalg.norm(p2 - p4)
    vertical2 = np.linalg.norm(p1 - p3)
    horizontal = np.linalg.norm(p0 - p3b)

    ear = (vertical1 + vertical2) / (2.0 * horizontal)
    return ear

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        h, w, _ = frame.shape
        landmarks = results.multi_face_landmarks[0].landmark

        # EAR for both eyes
        left_ear = eye_aspect_ratio(landmarks, LEFT_EYE, w, h)
        right_ear = eye_aspect_ratio(landmarks, RIGHT_EYE, w, h)
        avg_ear = (left_ear + right_ear) / 2.0

        # Threshold for eye open/close (tune between 0.2–0.3)
        if avg_ear < 0.25:
            status = "Eyes Closed"
            color = (0, 0, 255)
        else:
            status = "Eyes Open"
            color = (0, 255, 0)

        cv2.putText(frame, f"{status}", (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 2)

    cv2.imshow("Eye State Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#prototype 

import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)
    
    if not ret:
        break
    
    


    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(frame_rgb)

    if result.pose_landmarks:
        landmarks = result.pose_landmarks.landmark

        # Get left and right shoulder
        l_shoulder = np.array([
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y
        ])
        r_shoulder = np.array([
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y
        ])

        # Draw circles for shoulders
        h, w, _ = frame.shape
        cv2.circle(frame, (int(l_shoulder[0] * w), int(l_shoulder[1] * h)), 8, (0, 255, 0), -1)
        cv2.circle(frame, (int(r_shoulder[0] * w), int(r_shoulder[1] * h)), 8, (0, 255, 0), -1)

        # Calculate angle of shoulder line
        dx = r_shoulder[0] - l_shoulder[0]
        dy = r_shoulder[1] - l_shoulder[1]
        angle = np.degrees(np.arctan2(dy, dx))

        # Feedback
        if abs(angle) > 10:  # tolerance, straight ~0 degrees
            cv2.putText(frame, "Sit Straight!", (50, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            cv2.putText(frame, "Good Posture!", (50, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Shoulder Posture Feedback", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#prototype 

import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Pose model
pose = mp_pose.Pose(min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(frame_rgb)

    if result.pose_landmarks:
        h, w, _ = frame.shape

        # Draw full pose
        mp_drawing.draw_landmarks(
            frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS
        )

        # Extract shoulders
        left_shoulder = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]

        l_y = left_shoulder.y * h
        r_y = right_shoulder.y * h

        # Check alignment
        if abs(l_y - r_y) > 20:  # pixel difference threshold
            cv2.putText(frame, "Sit Straight!", (50, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            cv2.putText(frame, "Good Posture!", (50, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Posture Feedback", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#prototype 

import cv2
import mediapipe as mp
import numpy as np

# Initialize mediapipe modules
mp_pose = mp.solutions.pose
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# Function to check if eyes are open/closed using eye aspect ratio
def eye_aspect_ratio(landmarks, eye_idx, h, w):
    left = np.array([landmarks[eye_idx[0]].x * w, landmarks[eye_idx[0]].y * h])
    right = np.array([landmarks[eye_idx[3]].x * w, landmarks[eye_idx[3]].y * h])
    top = np.array([landmarks[eye_idx[1]].x * w, landmarks[eye_idx[1]].y * h])
    bottom = np.array([landmarks[eye_idx[2]].x * w, landmarks[eye_idx[2]].y * h])

    horiz = np.linalg.norm(left - right)
    vert = np.linalg.norm(top - bottom)
    return vert / horiz  # lower ratio = closed

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)
    if not ret:
        break

    h, w, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Pose detection (shoulders)
    pose_result = pose.process(frame_rgb)

    if pose_result.pose_landmarks:
        mp_drawing.draw_landmarks(frame, pose_result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        left_shoulder = pose_result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = pose_result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]

        l_y = left_shoulder.y * h
        r_y = right_shoulder.y * h

        if abs(l_y - r_y) > 20:  # difference in pixel height
            cv2.putText(frame, "Sit Straight!", (50, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            cv2.putText(frame, "Good Posture!", (50, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Face mesh (eyes)
    face_result = face_mesh.process(frame_rgb)

    if face_result.multi_face_landmarks:
        for face_landmarks in face_result.multi_face_landmarks:
            landmarks = face_landmarks.landmark

            # Indices for left and right eyes (Mediapipe standard)
            left_eye_idx = [33, 159, 145, 133]   # left, top, bottom, right
            right_eye_idx = [362, 386, 374, 263]

            left_ratio = eye_aspect_ratio(landmarks, left_eye_idx, h, w)
            right_ratio = eye_aspect_ratio(landmarks, right_eye_idx, h, w)

            avg_ratio = (left_ratio + right_ratio) / 2

            if avg_ratio < 0.2:  # tweak threshold
                cv2.putText(frame, "Eyes Closed - Stay Focused!", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(frame, "Eyes Open - Good!", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Posture & Eye Feedback", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#prototype 

import cv2
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

cap = cv2.VideoCapture(0)

# Iris landmark indices from Mediapipe
LEFT_IRIS = [474, 475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)
    if not ret:
        break

    h, w, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(frame_rgb)

    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            landmarks = face_landmarks.landmark

            # Get iris center for left & right eye
            left_iris = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in LEFT_IRIS]
            right_iris = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in RIGHT_IRIS]

            # Compute center of iris
            l_cx = sum([p[0] for p in left_iris]) // 4
            l_cy = sum([p[1] for p in left_iris]) // 4
            r_cx = sum([p[0] for p in right_iris]) // 4
            r_cy = sum([p[1] for p in right_iris]) // 4

            # Draw iris centers
            cv2.circle(frame, (l_cx, l_cy), 3, (0, 255, 0), -1)
            cv2.circle(frame, (r_cx, r_cy), 3, (0, 255, 0), -1)

            # Simple check: if iris roughly centered horizontally
            if w * 0.35 < l_cx < w * 0.65 and w * 0.35 < r_cx < w * 0.65:
                cv2.putText(frame, "Good Eye Contact!", (50, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Not Looking at Camera!", (50, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Eye Contact Feedback", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#prototype 

import cv2
import mediapipe as mp
import numpy as np

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# Iris landmark indices
LEFT_IRIS = [474, 475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

# Left & right eye boundary landmarks (Mediapipe reference indices)
LEFT_EYE = [33, 133, 159, 145]    # [left, right, top, bottom]
RIGHT_EYE = [362, 263, 386, 374]  # [left, right, top, bottom]

cap = cv2.VideoCapture(0)

def get_center(points):
    x = int(np.mean([p[0] for p in points]))
    y = int(np.mean([p[1] for p in points]))
    return (x, y)

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)
    if not ret:
        break

    h, w, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(frame_rgb)

    if result.multi_face_landmarks:
        landmarks = result.multi_face_landmarks[0].landmark

        # Get iris centers
        left_iris = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in LEFT_IRIS]
        right_iris = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in RIGHT_IRIS]
        l_center = get_center(left_iris)
        r_center = get_center(right_iris)

        # Get eye boundaries
        l_eye = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in LEFT_EYE]
        r_eye = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in RIGHT_EYE]

        # Draw eyes & iris
        cv2.circle(frame, l_center, 3, (0, 255, 0), -1)
        cv2.circle(frame, r_center, 3, (0, 255, 0), -1)

        # Horizontal + vertical check for left eye
        lx_min, lx_max = l_eye[0][0], l_eye[1][0]   # left & right
        ly_min, ly_max = l_eye[2][1], l_eye[3][1]   # top & bottom

        # Horizontal + vertical check for right eye
        rx_min, rx_max = r_eye[0][0], r_eye[1][0]
        ry_min, ry_max = r_eye[2][1], r_eye[3][1]

        # Check if iris is inside both eyes' bounding box
        left_inside = (lx_min < l_center[0] < lx_max) and (ly_min < l_center[1] < ly_max)
        right_inside = (rx_min < r_center[0] < rx_max) and (ry_min < r_center[1] < ry_max)

        if left_inside and right_inside:
            cv2.putText(frame, "Good Eye Contact!", (50, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Not Looking at Camera!", (50, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Eye Contact Feedback", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#prototype 

import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)
    if not ret:
        break

    h, w, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(frame_rgb)

    if result.pose_landmarks:
        mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Extract key points: nose + shoulders + ears
        landmarks = result.pose_landmarks.landmark
        nose = landmarks[mp_pose.PoseLandmark.NOSE]
        l_ear = landmarks[mp_pose.PoseLandmark.LEFT_EAR]
        r_ear = landmarks[mp_pose.PoseLandmark.RIGHT_EAR]

        # Convert to pixel coords
        nose = (int(nose.x * w), int(nose.y * h))
        l_ear = (int(l_ear.x * w), int(l_ear.y * h))
        r_ear = (int(r_ear.x * w), int(r_ear.y * h))

        # Draw points
        cv2.circle(frame, nose, 5, (0, 255, 0), -1)
        cv2.circle(frame, l_ear, 5, (255, 0, 0), -1)
        cv2.circle(frame, r_ear, 5, (255, 0, 0), -1)

        # Head tilt check → difference in ear height
        ear_diff = abs(l_ear[1] - r_ear[1])

        # Forward/backward tilt → distance between nose and ear line
        ear_mid_x = (l_ear[0] + r_ear[0]) // 2
        forward_tilt = abs(nose[0] - ear_mid_x)

        # Thresholds
        if ear_diff < 30 and forward_tilt < 50:  # tweak as needed
            cv2.putText(frame, "Good Head Position!", (50, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Adjust Head Position!", (50, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Head Position Feedback", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#prototype 

import cv2
import mediapipe as mp

# Mediapipe face mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
                                  max_num_faces=1,
                                  refine_landmarks=True,
                                  min_detection_confidence=0.5,
                                  min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape

            # Four key points
            forehead = landmarks.landmark[10]      # near top of forehead
            chin = landmarks.landmark[152]        # bottom of chin
            left_cheek = landmarks.landmark[234]  # left cheekbone
            right_cheek = landmarks.landmark[454] # right cheekbone

            # Convert to pixel coords
            f = (int(forehead.x * w), int(forehead.y * h))
            c = (int(chin.x * w), int(chin.y * h))
            l = (int(left_cheek.x * w), int(left_cheek.y * h))
            r = (int(right_cheek.x * w), int(right_cheek.y * h))

            # Draw points
            for point in [f, c, l, r]:
                cv2.circle(frame, point, 5, (0,255,0), -1)

            # Check vertical alignment forehead-chin
            vertical_alignment = abs(f[0] - c[0])  # X difference
            # Check left-right balance
            left_right_balance = abs((l[1] - r[1]))  # Y difference

            # Tolerances
            if vertical_alignment < 30 and left_right_balance < 20:
                text = "Good: Head straight"
                color = (0,255,0)
            else:
                text = "Not good: Sit straight"
                color = (0,0,255)

            cv2.putText(frame, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX,
                        1, color, 2)

    cv2.imshow("Head Alignment Check", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#prototype 

import cv2
import mediapipe as mp

# Face + Pose
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
                                  max_num_faces=1,
                                  refine_landmarks=True,
                                  min_detection_confidence=0.5,
                                  min_tracking_confidence=0.5)

pose = mp_pose.Pose(static_image_mode=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process with face + pose
    results_face = face_mesh.process(rgb)
    results_pose = pose.process(rgb)

    h, w, _ = frame.shape

    if results_face.multi_face_landmarks and results_pose.pose_landmarks:
        face_landmarks = results_face.multi_face_landmarks[0]
        pose_landmarks = results_pose.pose_landmarks.landmark

        # ---- FACE POINTS ----
        forehead = face_landmarks.landmark[10]
        chin = face_landmarks.landmark[152]
        left_cheek = face_landmarks.landmark[234]
        right_cheek = face_landmarks.landmark[454]

        f = (int(forehead.x * w), int(forehead.y * h))
        c = (int(chin.x * w), int(chin.y * h))
        l = (int(left_cheek.x * w), int(left_cheek.y * h))
        r = (int(right_cheek.x * w), int(right_cheek.y * h))

        # ---- CHEST MIDPOINT ----
        left_shoulder = pose_landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = pose_landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]

        chest_x = int((left_shoulder.x + right_shoulder.x) / 2 * w)
        chest_y = int((left_shoulder.y + right_shoulder.y) / 2 * h)
        chest = (chest_x, chest_y)

        # Draw landmarks
        for point in [f, c, l, r, chest]:
            cv2.circle(frame, point, 5, (0, 255, 0), -1)

        # ---- CHECK ALIGNMENTS ----
        vertical_alignment = abs(f[0] - c[0])   # forehead-chin x-diff
        cheek_balance = abs(l[1] - r[1])        # cheek height difference
        chin_chest_x = abs(c[0] - chest[0])     # chin vs chest alignment (X)
        chin_chest_y = abs(c[1] - chest[1])     # chin vs chest vertical gap

        # Tolerances
        if vertical_alignment < 25 and cheek_balance < 20 and chin_chest_x < 30 and 50 < chin_chest_y < 150:
            text = "Good: Head & Posture straight"
            color = (0, 255, 0)
        else:
            text = "Not good: Sit straight"
            color = (0, 0, 255)

        cv2.putText(frame, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    1, color, 2)

    cv2.imshow("Head + Posture Alignment", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [ ]:
#prototype 
import cv2
import mediapipe as mp
import math

# Face + Pose
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
                                  max_num_faces=1,
                                  refine_landmarks=True,
                                  min_detection_confidence=0.5,
                                  min_tracking_confidence=0.5)

pose = mp_pose.Pose(static_image_mode=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

baseline_chin_chest_dist = None  # Neutral reference distance

def euclidean(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process face + pose
    results_face = face_mesh.process(rgb)
    results_pose = pose.process(rgb)

    h, w, _ = frame.shape

    if results_face.multi_face_landmarks and results_pose.pose_landmarks:
        face_landmarks = results_face.multi_face_landmarks[0]
        pose_landmarks = results_pose.pose_landmarks.landmark

        # Face: chin
        chin = face_landmarks.landmark[152]
        c = (int(chin.x * w), int(chin.y * h))

        # Pose: chest midpoint from shoulders
        left_shoulder = pose_landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = pose_landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        chest_x = int((left_shoulder.x + right_shoulder.x) / 2 * w)
        chest_y = int((left_shoulder.y + right_shoulder.y) / 2 * h)
        chest = (chest_x, chest_y)

        # Draw points
        cv2.circle(frame, c, 5, (255, 0, 0), -1)
        cv2.circle(frame, chest, 5, (0, 255, 0), -1)
        cv2.line(frame, c, chest, (255, 255, 0), 2)

        # Distance
        chin_chest_dist = euclidean(c, chest)

        if baseline_chin_chest_dist is None:
            baseline_chin_chest_dist = chin_chest_dist  # set at start

        # Compare with baseline
        ratio = chin_chest_dist / baseline_chin_chest_dist

        if ratio > 1.25:  # 25% more → looking up
            feedback = "Not good: Looking up"
            color = (0, 0, 255)
        elif ratio < 0.75:  # 25% less → looking down
            feedback = "Not good: Looking down"
            color = (0, 0, 255)
        else:
            feedback = "Good: Head aligned"
            color = (0, 255, 0)

        cv2.putText(frame, feedback, (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    cv2.imshow("Head + Chest Alignment", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#-------------------------------------------------------- THE FINAL VERDICT ---------------------------------------------------------------------




import cv2
import mediapipe as mp
import math

# Face + Pose
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
                                  max_num_faces=1,
                                  refine_landmarks=True,
                                  min_detection_confidence=0.5,
                                  min_tracking_confidence=0.5)

pose = mp_pose.Pose(static_image_mode=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

baseline_chin_chest_dist = None  # Neutral reference distance

def euclidean(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process face + pose
    results_face = face_mesh.process(rgb)
    results_pose = pose.process(rgb)

    h, w, _ = frame.shape

    if results_face.multi_face_landmarks and results_pose.pose_landmarks:
        face_landmarks = results_face.multi_face_landmarks[0]
        pose_landmarks = results_pose.pose_landmarks.landmark

        # ---- FACE POINTS ----
        forehead = face_landmarks.landmark[10]
        chin = face_landmarks.landmark[152]
        left_cheek = face_landmarks.landmark[234]
        right_cheek = face_landmarks.landmark[454]

        f = (int(forehead.x * w), int(forehead.y * h))
        c = (int(chin.x * w), int(chin.y * h))
        l = (int(left_cheek.x * w), int(left_cheek.y * h))
        r = (int(right_cheek.x * w), int(right_cheek.y * h))

        # ---- SHOULDERS + CHEST ----
        left_shoulder = pose_landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = pose_landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]

        ls = (int(left_shoulder.x * w), int(left_shoulder.y * h))
        rs = (int(right_shoulder.x * w), int(right_shoulder.y * h))
        chest = (int((ls[0] + rs[0]) / 2), int((ls[1] + rs[1]) / 2))

        # Draw landmarks
        for pt in [f, c, l, r, ls, rs, chest]:
            cv2.circle(frame, pt, 5, (0, 255, 0), -1)

        # ---- HEAD STRAIGHT CHECK ----
        vertical_alignment = abs(f[0] - c[0])   # forehead-chin x-diff
        cheek_balance = abs(l[1] - r[1])        # cheek height difference

        head_good = vertical_alignment < 25 and cheek_balance < 20

        # ---- SHOULDER ALIGNMENT ----
        shoulder_balance = abs(ls[1] - rs[1])
        shoulders_good = shoulder_balance < 20

        # ---- CHIN–CHEST DISTANCE ----
        chin_chest_dist = euclidean(c, chest)
        if baseline_chin_chest_dist is None:
            baseline_chin_chest_dist = chin_chest_dist

        ratio = chin_chest_dist / baseline_chin_chest_dist
        if ratio > 1.25:
            chin_status = "Looking up"
            chin_good = False
        elif ratio < 0.75:
            chin_status = "Looking down"
            chin_good = False
        else:
            chin_status = "Head aligned"
            chin_good = True

        # ---- FINAL FEEDBACK ----
        feedback_lines = []
        if head_good:
            feedback_lines.append("GOOD HEAD POSITION")
        else:
            feedback_lines.append("NOT GOOD HEAD POSITION ")

        if shoulders_good:
            feedback_lines.append("GOOD SHOULDER ALLIGNMENT")
        else:
            feedback_lines.append("NOT GOOD SHOULDER ALIGNMENT")

        if chin_good:
            feedback_lines.append("GOOD " + chin_status)
        else:
            feedback_lines.append("NOT GOOD " + chin_status)

        # Display feedback
        y0 = 20
        for line in feedback_lines:
            cv2.putText(frame, line, (10, y0),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255) if "NOT GOOD" in line else (0, 255, 0), 2)
            y0 += 35

    cv2.imshow("Posture & Head Feedback", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
